In [12]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np

In [13]:
tf.keras.backend.clear_session()

In [14]:
SEQUENCE_LENGTH = 60
LSTM_SIZE = 128

In [15]:
embedding_matrix = np.load('./sentiment140_embedding.npy')

In [16]:
# not-exists word

embedding_matrix = np.pad(embedding_matrix, ((0,0),(0,1)), 'constant', constant_values=0)
embedding_matrix[-1][-1] = 1

In [21]:
tf.keras.backend.clear_session()
inputs = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,))
x = tf.keras.layers.Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False,mask_zero=True)(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LSTM_SIZE, return_sequences=True))(x)
# x = tf.keras.layers.SpatialDropout1D(0.2)(x)
x = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=200)(x, x)
x = tf.keras.layers.Dropout(0.15)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.15)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs, name='sentiment140_init_model')
model.summary()

Model: "sentiment140_init_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 60, 301)      8153187     ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 60, 256)      440320      ['embedding[0][0]']              
                                                                                                  
 multi_head_attention (MultiHea  (None, 60, 256)     821856      ['bidirectional[0][0]',          
 dAttention)                                                      'bidirecti

In [22]:
# tf.keras.backend.clear_session()
# inputs = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,))
# x = tf.keras.layers.Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False,mask_zero=True)(inputs)
# x = tf.keras.layers.SpatialDropout1D(0.2)(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LSTM_SIZE, return_sequences=True))(x)
# x = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=200)(x, x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(32, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
# model = tf.keras.Model(inputs=inputs, outputs=outputs, name='sentiment140_init_model')
# model.summary()

In [23]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tfa.optimizers.AdamW(weight_decay=1e-4),
    metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5)]
)

In [24]:
model.save('./sentiment140_init_model.h5')